# Monte Carlo AE and Amplitude Estimation Class

This notebook introduces two distinct Python classes:

1. `MCAE` class: Implemented in the module *montecarlo_ae* of the package **AE** in the library `QQuantLib` (**QQuantLib/AE/montecarlo_ae**). This class presents a Monte Carlo approach for computing amplitudes. The procedure estimates the amplitude of a specific state for a given oracle in a straightforward manner—by performing measurements and calculating the probability of obtaining the desired state.

2. `AE` class: Implemented in the module *ae_class* within the package **AE** of the library `QQuantLib` (**QQuantLib/AE/ae_class.py**). This class provides easy access to the various **Amplitude Estimation (AE)** algorithms implemented in the **AE** package.

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
#This cell loads the QLM solver. See notebook: 00_AboutTheNotebooksAndQPUs.ipynb
from QQuantLib.qpu.get_qpu import get_qpu
# myqlm qpus: python, c
# QLM qpus accessed using Qaptiva Access library: qlmass_linalg, qlmass_mps
# QLM qpus: Only in local Quantum Learning Machine: linalg, mps
my_qpus = ["python", "c", "qlmass_linalg", "qlmass_mps", "linalg", "mps"]

linalg_qpu = get_qpu(my_qpus[1])

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results
from QQuantLib.utils.utils import bitfield_to_int

## 0. Oracle generation

Before performing any amplitude estimation, we first need to load data into the quantum circuit. As this step is auxiliary and intended to demonstrate how the algorithm works, we will simply load a discrete probability distribution. 

In this example, we will use a quantum circuit with $ n = 3 $ qubits, which corresponds to a total of $ N = 2^n = 8 $ computational basis states. The discrete probability distribution we aim to load is defined as:

$$
p_d = \frac{(0, 1, 2, 3, 4, 5, 6, 7)}{0 + 1 + 2 + 3 + 4 + 5 + 6 + 7}.
$$

This distribution assigns probabilities proportional to the integers $ 0 $ through $ 7 $, normalized by their sum to ensure that the total probability equals 1.


In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)

Note that this probability distribution is properly normalised. For loading this probability into the quantum circuit we will use the function `load_probability` from **QQuantLib/DL/data_loading** module. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [ ]:
from QQuantLib.DL.data_loading import load_probability
oracle = load_probability(probability)

In [ ]:
%qatdisplay oracle --depth --svg

In [ ]:
#Information about the state selected by the oracle
target = [0,0,1]
index = [0,1,2]

In [ ]:
from QQuantLib.utils.utils import bitfield_to_int
desired_result = probability[bitfield_to_int(target)]
print("desired_result: {}".format(desired_result))

## 1. Monte Carlo Amplitude Estimation (MCAE)

In general, an amplitude estimation problem can be described as follows: given an oracle:
$$
\mathcal{O}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle + \sqrt{1-a}|\Psi_1\rangle,
$$
where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, our goal is to estimate $a$.

The `MCAE` class computes this $a$ amplitude in a straightforward manner. It creates the corresponding quantum circuit for the oracle, performs multiple measurements, and calculates the probability of the $|\Psi_0\rangle$ state.

### Inputs for Creating the Class

1. `Oracle`: A QLM `AbstractGate` or `QRoutine` that implements the Oracle.
2. `target`: The marked state in binary representation, provided as a Python list.
3. `index`: A list of qubits affected by the Oracle operator.

Additionally, a dictionary can be provided to configure the algorithm with the following optional keys:
- `qpu`: The QLM solver to be used (if not provided, `PyLinalg` will be used by default).
- `mcz_qlm`: A flag to use the QLM multi-controlled Z gate (`True`, default) or a multiplexor implementation (`False`).
- `shots`: An integer specifying the number of measurements to be performed on the quantum circuit.

In [ ]:
from QQuantLib.AE.montecarlo_ae import MCAE

In [ ]:
mc_dict = {
    #QPU
    'qpu': linalg_qpu,
    #Multicontrolate decomposition
    'mcz_qlm': False, 
    
    #shots
    'shots': 50,    
}

In [ ]:
mcae_object = MCAE(
    oracle=oracle,
    target=target,
    index=index,
    **mc_dict)

### `run` Method

To compute the amplitude of the target state, the **`run`** method should be used. This method returns the amplitude of the desired state, denoted as $ a $.

Additionally, the following object attributes are populated:

- `ae`: The amplitude of the desired state, $ a $.
- `ae_l`: The lower limit for $ a $, calculated as $ a_l = a - \frac{1}{\sqrt{\text{shots}}} $.
- `ae_u`: The upper limit for $ a $, calculated as $ a_u = a + \frac{1}{\sqrt{\text{shots}}} $.
- `schedule_pdf`: A pandas DataFrame containing the schedule used by the method. In this class, it represents the number of shots used for the computation.
- `oracle_calls`: The total number of oracle calls required for the complete execution of the algorithm.
- `max_oracle_depth`: The maximum number of applications of the oracle during the complete execution of the algorithm.

In [ ]:
mcae_object.run() 

In [ ]:
[mcae_object.ae_l, mcae_object.ae, mcae_object.ae_u]

In [ ]:
# upper limit
min(1.0, mcae_object.ae + 1.0 / np.sqrt(mc_dict["shots"]))

In [ ]:
# lower limit
max(0.0, mcae_object.ae - 1.0 / np.sqrt(mc_dict["shots"]))

In [ ]:
print("Absolute Error: {}".format(np.abs(mcae_object.ae - desired_result)))

For decreasing the error of the estimation more shots should be given

In [ ]:
mcae_object.shots = 100000
mcae_object.run() 
print("Absolute Error: {}".format(np.abs(mcae_object.ae - desired_result)))
print('Bounds for a: [mcae_object.ae_l, mcae_object.ae_u] = [{}, {}]'.format(mcae_object.ae_l, mcae_object.ae_u))

In [ ]:
mcae_object.schedule_pdf

In [ ]:
mcae_object.oracle_calls

In [ ]:
mcae_object.max_oracle_depth

In [ ]:
mcae_object.run_time

In [ ]:
mcae_object.quantum_time

## 2. `AE` class

The `AE` class allows the user to solve an **Amplitude Estimation** problem by providing and oracle and the type of **AE** algorithm user want to use.

### 2.1. The Amplitude Estimation Problem

In a general way, an amplitude estimation problem is the following: given an oracle:

$$\mathcal{O}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle +\sqrt{1-a}|\Psi_1\rangle,$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate $a$.  

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate $a$.  We can define an associated angle to $\sqrt{a}$ as $\sin^2{\theta} = a$, and the problem is thus rewritten as:
$$\mathcal{O}|0\rangle = |\Psi \rangle = \sin(\theta)|\Psi_0\rangle +\cos(\theta)|\Psi_1\rangle,$$

The foundation of any amplitude estimation algorithm is the Grover operator $\mathcal{Q}$, built onto the oracle $\mathcal{O}$, that has the following effect over our state $|\Psi\rangle$:

$$\mathcal{G}^{m}|\Psi\rangle = |\Psi \rangle = \sin\left((2m_k+1)\theta\right)|\Psi_0\rangle +\cos\left((2m_k+1)\theta\right)|\Psi_1\rangle,$$



### 2.2 Creating an Object from the AE Class

We have implemented a Python class called `AE` in the **QQuantLib/AE/ae_class** module, which allows users to access all amplitude estimation (AE) algorithms implemented in the **QQuantLib.AE** package in a transparent and user-friendly manner.

When creating an instance of the class, the following mandatory inputs **MUST** be provided:

1. `oracle`: A QLM `AbstractGate` or `QRoutine` that implements the Oracle for constructing the Grover operator.
2. `target`: The marked state in binary representation, provided as a Python list.
3. `index`: A list of qubits affected by the Grover operator.
4. `ae_type`: A string specifying the algorithm to use. Valid values include:
   - **MLAE**: For using the Maximum Likelihood Amplitude Estimation algorithm (see Notebook *03_Maximum_Likelihood_Amplitude_Estimation_Class.ipynb*).
   - **CQPEAE**: For using classical Phase Estimation with Quantum Fourier Transform (**QFT**) for Amplitude Estimation (see Notebook *04_Classical_Phase_Estimation_Class.ipynb*). It can also use windowed QPE (see Notebook *04-02_Classical_Phase_Estimation_Windows.ipynb*).
   - **IQPEAE**: For using classical Phase Estimation with iterative Quantum Fourier Transform (**QFT**) for Amplitude Estimation (see Notebook *05_Iterative_Quantum_Phase_Estimation_Class.ipynb*).
   - **IQAE**: For using Iterative Quantum Amplitude Estimation (see Notebook *06_Iterative_Quantum_Amplitude_Estimation_class.ipynb*). The modified **IQAE** can also be used by providing the string:
     - **mIQAE**
   - **RQAE**: For using Real Quantum Amplitude Estimation (see Notebook *07_Real_Quantum_Amplitude_Estimation_class.ipynb*). Variations of **RQAE** can also be used by providing the following strings:
     - **eRQAE**: Extended RQAE (see *07-02_Improvements_on_Real_Quantum_Amplitude_Estimation.ipynb*).
     - **mRQAE**: Modified RQAE (see *07-02_Improvements_on_Real_Quantum_Amplitude_Estimation.ipynb*).
     - **sRQAE**: RQAE with shots (see *07-02_Improvements_on_Real_Quantum_Amplitude_Estimation.ipynb*).
   - **MCAE**: For using the naive Monte Carlo approach for amplitude estimation (see the first part of the present notebook).

If any of the mandatory inputs (`Oracle`, `target`, or `index`) are missing, a `ValueError` is raised.

Additionally, other optional inputs can be provided as a Python dictionary to configure the selected algorithm. Refer to the respective notebooks for details on the parameters of each **AE** algorithm.

In [ ]:
from QQuantLib.AE.ae_class import AE

#### Example

To demonstrate how the `AE` class works, consider the following amplitude estimation problem:

$$
|\Psi\rangle = \mathcal{A}|0\rangle = \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[\sqrt{0}|0\rangle + \sqrt{1}|1\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right]. \tag{2}
$$

By comparing Equation (2) with Equation (1):

$$
\sqrt{a}|\Psi_0\rangle = \sin(\theta)|\Psi_0\rangle = \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle,
$$

and

$$
\sqrt{1-a}|\Psi_1\rangle = \cos(\theta)|\Psi_1\rangle = \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[\sqrt{0}|0\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right].
$$

In this case, the target state is $|1\rangle$, whose binary representation is $001$. This must be passed to the `target` variable as a list (`[0, 0, 1]`). Additionally, we need to provide the list of qubits where the operation is being performed. In this example, it is $[0, 1, 2]$, corresponding to the entire register.


In [ ]:
target = [0,0,1]
index = [0,1,2]

In [ ]:
from QQuantLib.utils.utils import bitfield_to_int
desired_result = probability[bitfield_to_int(target)]
print("desired_result: {}".format(desired_result))

The Python dictionary can be a very general one, allowing all parameters for configuring any algorithm to be provided. The class automatically selects the relevant parameters required by each specific algorithm. If a parameter is not provided for a particular method, the default parameter of that method will be used.

In [ ]:
m_k = [i for i in range(12)]
ae_dict = {
    #QPU
    'qpu': linalg_qpu,
    #Multi controlat decomposition
    'mcz_qlm': False, 
    
    #shots
    'shots': 100,
    
    #MLAE
    'schedule': [
        m_k,
        [100]*len(m_k)
    ],
    'delta' : 1.0e-6,
    'ns' : 10000,
    
    #CQPEAE
    'auxiliar_qbits_number': 10,
    'window' : None,
    'kaiser_alpha': None,
    
    #IQPEAE
    'cbits_number': 6,
    #IQAE & RQAQE
    'epsilon': 0.001,
    #IQAE
    'alpha': 0.05,
    #RQAE
    'gamma': 0.05,
    #'q': 1.2
}

In [ ]:
ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)

The *ae_type* parameter can not be provided initially and can be set in posterior way.

### 2.3 The `create_ae_solver` Method

The `create_ae_solver` method enables the user to instantiate the desired **AE** class. When this method is called, the following attributes are created:

- **`solver_dict`**: A Python dictionary containing the parameters for configuring the selected **AE** class (the class is determined by the `ae_type` attribute). If the selected **AE** class requires any parameter not included in the input dictionary, the default value of the class will be used.
  
- **`solver_ae`**: An object instantiated from one of the **AE** classes in the **QQuantLib.AE** package (the class is determined by the `ae_type` attribute). The configuration of this object is based on the parameters provided in the `solver_dict`.

If the `ae_type` attribute is set to `None`, the `create_ae_solver` method will raise an error.


In [ ]:
#ae_type is None so this cell raises an error
ae_object.create_ae_solver()

In [ ]:
#this is None
ae_object.solver_dict

In [ ]:
#Now we select the MLAE method
ae_object.ae_type = 'MLAE'
ae_object.create_ae_solver()

In [ ]:
#Configuration dictionary for the AE class
ae_object.solver_dict

In [ ]:
#For configuring the parameters of the method the keys from the solver_dict will be used
print('From the method: {}'.format(ae_object.solver_ae.schedule))

print('From the solver_dict attribute: {}'.format(ae_object.solver_dict['schedule']))

We can change the **AE** algorithm to use

In [ ]:
ae_object.ae_type = 'IQPEAE'
ae_object.create_ae_solver()

In [ ]:
ae_object.solver_ae

In [ ]:
#Configuration dictionary for the new AE selected class
ae_object.solver_dict

In [ ]:
ae_object.ae_type = 'RQAE'
ae_object.create_ae_solver()
#In this case the q parameter is not provided in the input dictionary so it doesn't appear in the solver_dict
ae_object.solver_dict

In [ ]:
#The default value for the ae class will be used instead:
ae_object.solver_ae.ratio

### 2.4 Executing the Algorithm

The `solver_ae` attribute is an object of the desired **AE** class, so its `run` method can be used directly to execute the algorithm.

**BE AWARE!!**  
This is not the recommended way to use the **AE** class. The **AE** class has its own `run` method, which is the preferred and recommended way to execute the algorithm (see Section 3). This section is included for pedagogical purposes only.

In [ ]:
m_k = [i for i in range(12)]
ae_dict = {
    #QPU
    'qpu': linalg_qpu,
    #Multi controlled decomposition
    'mcz_qlm': False, 
    
    #shots
    'shots': 100,
    
    #MLAE
    'schedule': [
        m_k,
        [100]*len(m_k)
    ],
    'delta' : 1.0e-6,
    'ns' : 10000,
    
    #CQPEAE
    'auxiliar_qbits_number': 10,
    'window' : None,
    'kaiser_alpha': None,    
    #IQPEAE
    'cbits_number': 6,
    #IQAE & RQAQE
    'epsilon': 0.001,
    #IQAE
    'alpha': 0.05,
    #RQAE
    'gamma': 0.05,
    'q': 1.2
}

In [ ]:
ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)

In [ ]:
ae_object.ae_type = 'MLAE'
ae_object.create_ae_solver()

In [ ]:
ae_object.solver_ae.run()

Additionally all the attributes of the class are available:

In [ ]:
ae_object.solver_ae.h_k

In [ ]:
ae_object.solver_ae.oracle_calls

In [ ]:
ae_object.solver_ae.schedule_pdf

We can change now the **AE** algorithm

In [ ]:
ae_object.ae_type = 'IQAE'
ae_object.create_ae_solver()
ae_object.solver_ae.run()

Again the particular attributes of the solver can be accessed in an easy way

In [ ]:
[ae_object.solver_ae.ae, ae_object.solver_ae.ae_l, ae_object.solver_ae.ae_u]

In [ ]:
ae_object.solver_ae.oracle_calls

In [ ]:
ae_object.solver_ae.schedule_pdf

## 3. AE complete execution

### BE AWARE
-----------------

In section 2 the complete **AE** class was explained in a detailed way for pedagogical purposes for understanding what is the design of the **AE** class. The users  **SHOULD NOT** use the procedures provided in section 2 for  working with the class. Instead, Section 3 shows how the user **SHOULD** interact with the **AE** class.

----

The **AE** class has a *run* method that executes, in a transparent way, the *run* method of the solver object and does some post-process over the obtained results. The following attributes are populated when the method is executed:

* *ae_pdf*: pandas DataFrame with the direct result of the ae. This data frame has 3 columns:
    * ae: is the desired $\sqrt{a}$
    * ae_l: is the lower bound for the $\sqrt{a}$. **MLAE** does not provide upper and lower bounds so this value will be equal to *ae*.
    * ae_u: is the upper boudn for the $\sqrt{a}$.  **MLAE** does not provide upper and lower bounds so this value will be equal to *ae*.
* *oracle_calls*: number of total oracle calls for a complete execution of the algorithm
* *max_oracle_depth*: maximum number of applications of the oracle for the complete execution of the algorithm.
* *schedule_pdf*: an attribute that depends on the AE method:
    * CQPEAE and IQPEAE: None.
    * IQAE, RQAE: pandas DataFrame with the times the Grover-like operator was applied (m_k column) and the correspondent number of shots (shots column)
    * MLAE: pandas DataFrame with the times the Grover-like operator was applied (m_k column) the correspondent number of shots (n_k column) and the number of positive outcomes obtained (h_k)
    * MCAE: pandas DataFrame with the number of shots used.

In [ ]:
# Base Dictionary
ae_dict = {
    #QPU
    'qpu': linalg_qpu,
    #Multi controlled decomposition
    'mcz_qlm': False, 
}

### 3.1 MLAE

In [ ]:
# Update with MLAE configuration
m_k = [i for i in range(12)]
ae_dict.update({
    "ae_type":'MLAE',
    #MLAE
    'schedule': [
        m_k,
        [100]*len(m_k)
    ],
    'delta' : 1.0e-6,
    'ns' : 10000,    
})

ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)
ae_object.run()

In [ ]:
mlae_result = ae_object.ae_pdf
mlae_result

In [ ]:
print("Absolute Error for MLAE: {}".format(np.abs(mlae_result['ae'][0] - desired_result)))

In [ ]:
ae_object.oracle_calls

In [ ]:
ae_object.max_oracle_depth

In [ ]:
ae_object.schedule_pdf

In [ ]:
print(ae_object.run_time)
print(ae_object.quantum_time)

### 3.2 CQPEAE

In [ ]:
# Update with CQPEAE configuration
ae_dict.update({
    "ae_type":'CQPEAE',
    'shots': 10000,
    'auxiliar_qbits_number': 10,
    'window' : None,
    'kaiser_alpha': None,    
})
ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)
ae_object.run()
cqpeae_result = ae_object.ae_pdf

In [ ]:
cqpeae_result

In [ ]:
print("Absolute Error for CQPEAE: {}".format(
    np.abs(cqpeae_result['ae'][0] - desired_result)))

In [ ]:
ae_object.oracle_calls

In [ ]:
ae_object.max_oracle_depth

In [ ]:
print(ae_object.run_time)
print(ae_object.quantum_time)

#### 3.2.1 Window QPE

The window **QPE** initialization of the auxiliary qubits can be provided easily

In [ ]:
ae_dict.update({
    "ae_type":'CQPEAE',
    'auxiliar_qbits_number': 10,
    'shots': 10000,
    "window": "sine",
    'kaiser_alpha': None,  
})


ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)
ae_object.run()
cqpeae_result = ae_object.ae_pdf

print("Absolute Error for CQPEAE: {}".format(
    np.abs(cqpeae_result['ae'][0] - desired_result)))

In [ ]:
ae_dict.update({
    "ae_type":'CQPEAE',
    'auxiliar_qbits_number': 10,
    'shots': 10000,
    "window": "Kaiser",
    'kaiser_alpha': 2,  
})


ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)
ae_object.run()
cqpeae_result = ae_object.ae_pdf

print("Absolute Error for CQPEAE: {}".format(
    np.abs(cqpeae_result['ae'][0] - desired_result)))

### 3.3 IQPEAE

In [ ]:
ae_dict.update({
    "ae_type":'IQPEAE',
    'cbits_number': 6,
    'shots': 10,
})
ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)
ae_object.run()
iqpeae_result = ae_object.ae_pdf

In [ ]:
iqpeae_result

In [ ]:
print("Absolute Error for IQPEAE: {}".format(np.abs(iqpeae_result['ae'][0] - desired_result)))

In [ ]:
ae_object.oracle_calls

In [ ]:
ae_object.max_oracle_depth

In [ ]:
ae_object.schedule_pdf

In [ ]:
print(ae_object.run_time)
print(ae_object.quantum_time)

### 3.4 IQAE

In [ ]:
ae_dict.update({
    "ae_type":'IQAE',
    'epsilon': 0.0001,
    'alpha': 0.05,
    'shots': 1000,
})
ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)
ae_object.run()
iqae_result = ae_object.ae_pdf

In [ ]:
iqae_result

In [ ]:
print("Absolute Error for IQAE: {}".format(np.abs(iqae_result['ae'][0] - desired_result)))

In [ ]:
ae_object.oracle_calls

In [ ]:
ae_object.max_oracle_depth

In [ ]:
ae_object.schedule_pdf

In [ ]:
print(ae_object.run_time)
print(ae_object.quantum_time)

**mIQAE** can be used easily

In [ ]:
ae_dict.update({
    "ae_type":'mIQAE',
    'epsilon': 0.0001,
    'alpha': 0.05,
    'shots': 1000,
})
ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)
ae_object.run()
miqae_result = ae_object.ae_pdf

In [ ]:
print("Absolute Error for IQAE: {}".format(np.abs(miqae_result['ae'][0] - desired_result)))

In [ ]:
ae_object.oracle_calls

In [ ]:
ae_object.schedule_pdf

### 3.5 RQAE

### BE AWARE
-----------------

The RQAE algorithm provides directly the **Amplitude** of the state so we need to compare vs the square root! This is applied to the **RQAE** variations (i.e. **mRQAE**, **sRQAE** and **eRQAE**)

$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & a|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \sqrt{1-a^2}|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$

In [ ]:
ae_dict.update({
    "ae_type":'RQAE',
    'epsilon': 0.0001,
    'gamma': 0.05,
    'q': 2,
})
ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)
ae_object.run()
ae_object.run()
rqae_result = ae_object.ae_pdf

In [ ]:
rqae_result**2

in this case the desired result is 

In [ ]:
print("Absolute Error for RQAE: {}".format(np.abs(rqae_result['ae'][0]**2 -desired_result)))

In [ ]:
ae_object.oracle_calls

In [ ]:
ae_object.max_oracle_depth

In [ ]:
ae_object.schedule_pdf

In [ ]:
print(ae_object.run_time)
print(ae_object.quantum_time)

**mRQAE** can be used easily

In [ ]:
ae_dict.update({
    "ae_type":'mRQAE',
    'epsilon': 0.0001,
    'gamma': 0.05,
    'q': 2,
})
ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)
ae_object.run()
ae_object.run()
mrqae_result = ae_object.ae_pdf

In [ ]:
print("Absolute Error for mRQAE: {}".format(np.abs(mrqae_result['ae'][0]**2 -desired_result)))

In [ ]:
ae_object.oracle_calls

In [ ]:
ae_object.max_oracle_depth

In [ ]:
ae_object.schedule_pdf

### 3.6 MCAE

In [ ]:
ae_dict.update({
    "ae_type":'MCAE',
    'shots': 1000
})
ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)

ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)


ae_object.run()
mcae_result = ae_object.ae_pdf

In [ ]:
print("Absolute Error for MCAE: {}".format(np.abs(mcae_result['ae'][0] - desired_result)))

In [ ]:
mcae_result

In [ ]:
ae_object.oracle_calls

In [ ]:
ae_object.max_oracle_depth

In [ ]:
ae_object.schedule_pdf

In [ ]:
print(ae_object.run_time)
print(ae_object.quantum_time)

## 4. Amplitude Estimation Algorithms comparison

To compare the performance of the different **AE** algorithms implemented in the **AE** class the package **benchmark.compare_ae_probability.probability_estimation** inside of the **FinancialApplications** software package has been developed. In this module a probability density distribution is loaded into a quantum state and different **AE** algorithms can be used for estimating the amplitude of fixed input state. We refer to the notebook  *CompareAEalgorithmsOnPureProbability.ipynb* (in **benchmark.compare_ae_probability** package) for more info.